In [ ]:
import dask.dataframe as dask_pd
import dask_geopandas as dask_gpd
import geopandas as gpd
from dask.distributed import Client

In [ ]:
# explore = dask_pd.read_csv("sample-folder/ue_traffic.csv", usecols=[0,1,2,3,4,10,12,13], assume_missing=True)
explore = dask_pd.read_csv("sample-folder/ue_traffic.csv", assume_missing=True)
cellrebel = dask_pd.read_csv("", assume_missing=True)

In [ ]:
explore = explore[explore['ue_throughput_dl_drb_kbps'] > 0] 

In [ ]:
explore.head(10)

In [ ]:
split = explore['hour'].str.split(pat='.', expand=True, n=1)

In [ ]:
split = split.rename(columns={0:"Hour + 7"})

In [ ]:
explore = dask_pd.concat([explore, split['Hour + 7']], axis=1,join='inner', ignore_unknown_divisions=True)

In [ ]:
explore.head(10)

In [ ]:
explore['Hour + 7'] = explore['Hour + 7'].astype(int) + 7

In [ ]:
explore.head(20)